My medium : https://medium.com/@pawito236/thai-g2p-seq-to-seq-a1eb96c3be18

My Github : https://github.com/pawito236/Thai_G2P_seq-to-seg

Reference : https://medium.com/nectec/g2p-sequence-to-sequence-53a843d6702a

# Lib

In [1]:
!git clone https://github.com/cmusphinx/g2p-seq2seq.git
%cd g2p-seq2seq/
!pip install tensor2tensor==1.7.0
!python setup.py install
!!pip install tensorflow==1.13.1
!pip install tensorflow-gpu==1.13.1

Cloning into 'g2p-seq2seq'...
remote: Enumerating objects: 1406, done.
remote: Total 1406 (delta 0), reused 0 (delta 0), pack-reused 1406
Receiving objects: 100% (1406/1406), 884.85 KiB | 11.64 MiB/s, done.
Resolving deltas: 100% (906/906), done.
/content/g2p-seq2seq
     |████████████████████████████████| 853 kB 9.8 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 5.8 MB 72.6 MB/s 
     |████████████████████████████████| 251 kB 57.8 MB/s 
  Created wheel for bz2file: filename=bz2file-0.98-py3-none-any.whl size=6882 sha256=1ec9afeb927471f1565914e11ca009d232914eb82f45e3e98a2a4ad376854a24
  Stored in directory: /root/.cache/pip/wheels/85/ce/8d/b5f76b602b16a8a39f2ded74189cf5f09fc4a87bea16c54a8b
Successfully built bz2file
running install
running bdist_egg
running egg_info
creating g2p_seq2seq.egg-info
writing g2p_seq2seq.egg-info/PKG-INFO
writing dependency_links to g2p_seq2seq.egg-info/dependency_links.txt
writing entry points to g2p_se

In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-c070cdb2-ff0a-39d3-69f4-852560d2cee6)


# Extracting the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import shutil
def download():
    url = "https://github.com/korakot/corpus/releases/download/v1.0/AIFORTHAI-TSync2Corpus.zip"
    print("NECTEC licensed TSync2 under CC-BY-NC-SA")
    print("Start downloading: .. ")
    os.system(f"wget {url}")
    os.system("unzip AIFORTHAI-TSync2Corpus.zip")
    os.system("rm AIFORTHAI-TSync2Corpus.zip")
    print("Finished")

In [4]:
download()

NECTEC licensed TSync2 under CC-BY-NC-SA
Start downloading: .. 
Finished


# Prepare phoneme Training data


In [5]:
import re
my_string = "ก็|ต่าง|ปรับ|สี|สัน|ให้|อินเทรนด์|บู๊ทส์|รีเทล|จำกัด|"
pm = 'k-@@-z^-2|*t-aa-ng^-1|*pr-a-p^-1|*s-ii-z^-4|*s-a-n^-4|*h-a-j^-2|*z-i-n^-0|thr-ee-n^-0|*b-uu-t^-3|*r-ii-z^-0|th-ee-l-0|*c-a-m^-0|k-a-t^-1|*'
print(len(re.findall("\|", my_string)))
print(len(re.findall("\|", pm)))

10
13


In [6]:
pm.replace('*','').split("|")[0].split("-")

['k', '@@', 'z^', '2']

In [7]:
import glob
import re
# Get phoneme from txt file
def linePm(inputPath):
  c = 0
  data = []
  fileList = glob.glob(inputPath+'/wrd_ph/*.txt')
  for fileName in fileList:
    # print("in loop")
    # Using readlines()
    file1 = open(fileName, 'r')
    Lines = file1.readlines()
    sent = Lines[0]
    pm = Lines[-1][:-2]
    word_len = len(re.findall("\|", sent))
    pm_len = len(re.findall("\|", pm))

    # Check if the same len word
    if(word_len != pm_len):
      continue
    c = c+1
    wrdArr = sent.split("|")
    pmArr = pm.replace('*','').split('|')
    for n in range(word_len):
      pmStr = ''
      for pm in pmArr[n].split("-"):
        if pm != ' ':
          pmStr = pmStr + pm + ' '
      data.append("{} {}".format(wrdArr[n], pmStr.strip()))
  print("Total readable file :", c)
  return(data)


In [8]:
dataSet = linePm('/content/g2p-seq2seq/TSync2')

Total readable file : 280


In [9]:
len(dataSet)

2860

In [ ]:
dataSet

In [10]:
import os
import glob
import tqdm
import multiprocessing
from sklearn.model_selection import train_test_split

In [11]:
# Write Nex Training and testing file
# format written will be : /content/wav/wav000.wav|th-a-ng^-3 j-a-ng^-0 ch-uua-n^-0 p-a-j^-0 n-a-ng^-2 d-vv-m^-1 l-a-w^-2 th-xx-w^-4 p-a-m^-3 kh-aa-l^-0 th-e-k^-1

trainSet, testSet = train_test_split(dataSet, test_size=0.05, random_state=42)
print("Count trainSet: ", len(trainSet))
print("Count testSet: ", len(testSet))
with open(os.path.join('TSync2', 'g2p_train.dict'), 'w') as fw_train:
    for dataTrain in trainSet:
        if dataTrain!='' and dataTrain is not None:
            fw_train.write(dataTrain+'\n')
with open(os.path.join('TSync2', 'g2p_test.dict'), 'w') as fw_test:
    for dataTest in testSet:
        if dataTest!='' and dataTest is not None:
            fw_test.write(dataTest+'\n')

Count trainSet:  2717
Count testSet:  143


In [20]:
!mkdir /content/g2p-seq2seq/data

mkdir: cannot create directory ‘/content/g2p-seq2seq/data’: File exists


In [21]:
!cp /content/g2p-seq2seq/TSync2/g2p_train.dict /content/g2p-seq2seq/data
!cp /content/g2p-seq2seq/TSync2/g2p_test.dict /content/g2p-seq2seq/data

# Train

In [18]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In params.py ADD : self.gpu_order = "0"

In [22]:
!mkdir model 
!mkdir output
!g2p-seq2seq --train data/g2p_train.dict --model_dir model/
# !g2p-seq2seq --decode data/g2p_test.dict  --model_dir model/ --output output/result_seq2seq.txt

mkdir: cannot create directory ‘model’: File exists
mkdir: cannot create directory ‘output’: File exists
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/li

In [25]:
%%timeit
!g2p-seq2seq --decode data/g2p_test.dict  --model_dir model/ --output output/result_seq2seq.txt

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [26]:
!zip -r /content/model.zip /content/g2p-seq2seq/model/

  adding: content/g2p-seq2seq/model/ (stored 0%)
  adding: content/g2p-seq2seq/model/model.ckpt-4000.data-00001-of-00002 (deflated 9%)
  adding: content/g2p-seq2seq/model/events.out.tfevents.1649925809.ae07f7adf9d5 (deflated 95%)
  adding: content/g2p-seq2seq/model/model.ckpt-4000.data-00000-of-00002 (deflated 50%)
  adding: content/g2p-seq2seq/model/graph.pbtxt (deflated 97%)
  adding: content/g2p-seq2seq/model/vocab.g2p (deflated 41%)
  adding: content/g2p-seq2seq/model/eval.preprocessed (deflated 85%)
  adding: content/g2p-seq2seq/model/checkpoint (deflated 43%)
  adding: content/g2p-seq2seq/model/eval/ (stored 0%)
  adding: content/g2p-seq2seq/model/eval/events.out.tfevents.1649925937.ae07f7adf9d5 (deflated 93%)
  adding: content/g2p-seq2seq/model/model.ckpt-4000.index (deflated 73%)
  adding: content/g2p-seq2seq/model/train.preprocessed (deflated 87%)
  adding: content/g2p-seq2seq/model/model.ckpt-4000.meta (deflated 95%)
  adding: content/g2p-seq2seq/model/model.params (deflated 